In [ ]:
import os
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import chromadb

current_dir = os.path.dirname(os.path.abspath("__file__"))
file_path = os.path.join(current_dir, "books", "智慧型手機使用手冊.txt")
persistent_directory = os.path.join(current_dir, "db", "chroma_db_e5")

if not os.path.exists(persistent_directory):
    print("持久化目錄不存在,正在初始化向量資料庫...")

    #確保文字檔案存在
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"檔案{file_path}不存在。請檢查路徑.")


    loader = TextLoader(file_path)
    documents = loader.load()
    
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)

    print(docs[0].page_content)

    embeddings = HuggingFaceEmbeddings(
        model_name='intfloat/multilingual-e5-large'
    )
    print("\n完成下載")

    # 使用PersistentClient 以避免權限問題
    client = chromadb.PersistentClient(path=persistent_directory)

    db = Chroma.from_documents(
        docs,
        embeddings,
        client=client,
        collection_name="smartphone_manual"
    )
    print("\n--完成建立向量存儲 ---")
else:
    print("向量資料庫已存在,無需初始化。")

In [ ]:
import os
current_dir = os.path.dirname(os.path.abspath('__file__'))
persistent_directory = os.path.join(current_dir, "db", "chroma_db_e5")

embeddings = HuggingFaceEmbeddings(
    model_name = "intfloat/multilingual-e5-large"
)

client = chromadb.PersistentClient(path=persistent_directory)
db = Chroma(
    client=client,
    collection_name= "smartphone_manual",
    embedding_function=embeddings
)

query = "如何設定指紋辦識?"

retriever = db.as_retriever(
    search_type='similarity',
    search_kwargs={"k":3}
)

relevant_docs = retriever.invoke(query)

for i,doc in enumerate(relevant_docs, 1):
    print(f"文件{i}:\n{doc.page_content}\n")
    if doc.metadata:
        print(f"來源:{doc.metadata.get('source','Unknown')}\n")

In [6]:
# 定義使用者的問題
question = "如何設定指紋辨識?"

print(f"問題:{question}\n")

print("=" * 60)
print("步驟 1: 從向量資料庫檢索相關文件")
print("=" * 60)

retriever = db.as_retriever(
    search_type='similarity',
    search_kwargs={"k": 2} # 只取最相關的2個文件
)

retriever_docs = retriever.invoke(question)

print(f"找到{len(retriever_docs)}相關文件\n")
for i, doc in enumerate(retriever_docs,1):
    print(f"文件{i}(前100字)")
    print(doc.page_content[:100] + "...\n")


問題:如何設定指紋辨識?

步驟 1: 從向量資料庫檢索相關文件
找到2相關文件

文件1(前100字)
6.3 通知管理
設定 > 通知

管理應用程式通知:
1. 選擇應用程式
2. 開啟/關閉通知
3. 設定通知類別
4. 通知顯示方式(橫幅/鎖定螢幕/通知中心)
5. 通知聲音與震動

勿擾模式:...

文件2(前100字)
智慧型手機使用手冊

歡迎使用您的新智慧型手機!本手冊將協助您快速上手並充分利用手機的各項功能。

第一章：開始使用

1.1 開箱與首次設定
打開包裝盒後,您會看到以下配件:
- 智慧型手機主機 x...



In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import ChatOllama

# 定義提示模板
template = """你是一個智慧型手機的客服助手。請根據以下參考資料回答使用者的問題。

參考資料：
{context}

使用者問題：{question}

請用繁體中文回答，並且：
1. 只根據參考資料回答，不要編造內容
2. 如果參考資料中沒有答案，請誠實說「我在資料中找不到相關資訊」
3. 回答要清楚、具體、有條理

回答："""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOllama(model="llama3.2:latest", temperature=0)

# 定義文件格式化函數
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

# 建立完整的 RAG Chain
rag_chain = (
    {"context":retriever | format_docs, "question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser
)

